# Pretraživanje informacija

Retke matrice se pojavljuju u mnogim domenima. Sistemi preporuka, obrada teksta, kompjuterska vizija ili grafovska modelovanja su samo neki od njih. U nastavku će biti prikazan jedan pojednostavljen sistem za pretraživanje informacija koji za zadati upit korisnika treba da pronađe najrelevantniji dokument kolekcije dokumenata. <img src='assets/ir_system.png' style='height: 300px;'>

Za razliku od reprezentacije teksta koju smo koristiti u zadatku sumarizacije, u ovom problemu ćemo za reprezentaciju dokumenata i upita koristiti Tf-Idf vektorizacije. 

### TF-IDF reprezentacija

Prvi korak u obradi kolekcije dokumenata je kreiranje `vokabulara`. Ovaj zadatak podrazumeva izdvajanje svih reči koje se nalaze u kolekciji dokumenata i njihovo leksikografsko sortiranje. U realnim zadacima, u zavisnosti od kolekcije sa kojom se radi, veličina vokabulara može varirati od 10000 do 100000 reči. Da bi se redukovala dimenzija vokabulara, samo izdvajenje reči može da prati filtriranje dopustivih reči, na primer, izbacivanje brojeva, interpunkcije, reči koje se retko pojavljuju poput pravopisnih propusta, reči koje se često pojavljuju poput članova ili predloga. Takođe, izdvajanje reči može pratiti i njihova normalizacija. Na primer, reči *playing*, *played* i *plays* se mogu svesti na istu lingvističku formu (lemu) *play* ili na neki drugi veštački koren (engl. stem), na primer *pla*, koji bi ukazao na zajedničko značenje. 

Svaki dokument polazne kolekcije je dalje potrebno izraziti vektorski u terminima izdvojenog vokabulara tako što se na pozicijima koje odgovaraju rečima koje sadrže upisuju odgovorajuće težine. Na ostalim pozicijama se upisuju nule pa su dobijene reprezentacije dokumenata vrlo retke. 

Težine koje odgovaraju pojedinačnim rečima se mogu računati na razne načine. TF-IDF reprezentacija očekuje da se u obzir uzme frekvencija same reči (engl. term frequency) u dokumentu, kao i njena inverzna frekvencija dokumenta (engl. inverse document frequency) kojom se ukazuje na prisutnost reči u celoj kolekciji dokumenata. 

Težina *tf-idf* se računa kao proizvod faktora $Tf(t, d)$ i $Idf(t, D)$.

$Tf(t, d)$ predstavlja količnik broja pojavljivanja reči/termina $t$ u dokumentu $d$ i ukupnog broja reči u dokumentu $d$ i računa se kao
$$Tf(t, d) = \frac{count(t)}{|d|}$$

$Idf(t, D)$ predstavlja logaritam količnika ukupnog broja dokumenata skupa $D$ i broja dokumenata (iz skupa 
$D$) u kojima se pojavljuje reč/termin $t$ i računa se kao
$$Idf(t, D) = log(\frac{|D|}{N_t})$$


$$TfIdf(t, d, D) =Tf(t, d)\cdot Idf(t, D)$$

Važnost termina (reči) raste sa rastom broja njegovog pojavljivanja u dokumentu, ali se otežava njegovim ukupnim brojem pojavljivanja u celokupnom skupu dokumenata (rečima koje se pojavljuju u većem broju dokumenata pridružuje se manja vrednost jer je i njihova diskriminativnost manja).

Postoje i mnoge varijacije na ovu temu o kojima se može više pročitati u članku na [Vikipediji](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) ili u [prezentaciji](https://slideplayer.com/slide/11565250/) koja prati sadržaj knjige *Natural Language and Speech Processing* autora Dana Džurafskog. 

In [1]:
import numpy as np
from scipy import sparse

Mi ćemo za računanje ovakvih reprezentacija iskoristiti podršku `scikit-learn` biblioteke i njenu klasu `TfIdVectorizer`. 

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

Kolekcija dokumenata sa kojom ćemo raditi je poznata pod imenom `20newsgroup`. Ona sadrži kolekciju 18000 *newsgroup* postova razvrstanih u 20 kategorija. Mi ćemo u radu upotrebiti dve kategorije, `sci.space` i `comp.graphics`.

In [5]:
from sklearn.datasets import fetch_20newsgroups

`TfIdfVectorizer` će za zadatu kolekciju kreirati Tf-Idf matricu. Opciono mu se mogu proslediti parametri podešavanja, na primer, mi ćemo isključiti sve reči koje imaju manje od 5 pojavljivanja (parametar `min_df` sa vrednošću 5), sve reči ćemo svesti na zapis malim slovima (parametar `lowercase` sa podrazumevanom vrednošću True), isključićemo stop reči za engleski jezik (parametar `stop_words` sa vrednošću english), a pod rečima ćemo smatrati niske karaktera opisane regularnim izrazom \\b\\w\\w+\\b (podrazumevana vrednost `token_pattern` parametra).